# Example of 1D particle in cell code for plasma dynamics

We set up some test problems involving ions and "electrons". 

In [1]:
import numpy as np

class PICSimulation:
    """
    A simple class-based 1D PIC simulation.
    Advances current particle positions, velocities, time, etc.,
    The run() method returns a list of snapshots for later visualization
    """

    def __init__(self, xp, vp, xe, ve, Ngrid, dt,
                 massIon, massE, chargeIon, chargeE):
        """
        Initialize the simulation with initial arrays and parameters.

        xp, vp: arrays of proton positions & velocities
        xe, ve: arrays of electron positions & velocities
        Ngrid:  number of grid cells
        dt:     time step
        massIon, massE, chargeIon, chargeE: physical parameters
        """
        # Particle arrays (1D)
        self.xp = xp   # shape (N_ion,) 
        self.vp = vp
        self.xe = xe
        self.ve = ve

        # Grid info
        self.Ngrid = Ngrid
        self.dx = 1.0 / Ngrid

        # Physical parameters
        self.dt = dt
        self.massIon = massIon
        self.massE   = massE
        self.chargeIon = chargeIon
        self.chargeE   = chargeE

        # Simulation time/step
        self.time = 0.0
        self.step_count = 0

    def array_periodic_boundary(self, x):
        """Wrap positions x into [0,1) to enforce periodic boundary."""
        x %= 1

    def CIC_deposit(self, x, weight=1.0):
        """Simple Python version of CIC deposit."""
        rho = np.zeros(self.Ngrid)
        m = weight / x.size

        left = x - 0.5 * self.dx
        xi = np.floor(left / self.dx).astype(np.int32) % self.Ngrid
        frac = 1.0 - (left / self.dx - xi) % 1

        # neighbor cell
        xir = (xi + 1) % self.Ngrid
        
        # main deposit
        w_main = frac * m
        rho_main = np.bincount(xi, weights=w_main, minlength=self.Ngrid)

        # neighbor deposit
        w_next = (1.0 - frac) * m
        rho_next = np.bincount(xir, weights=w_next, minlength=self.Ngrid)

        rho = rho_main + rho_next
        return rho * self.Ngrid
    
    def rebin_grad_phi(self, x, grad_phi):
        """
        Interpolate grad_phi from grid to particle positions x (CIC gather).
        """
        left = x - 0.5 * self.dx
        xi = np.floor(left / self.dx).astype(np.int32) % self.Ngrid
        frac = 1.0 - (left / self.dx - xi) % 1  # Ensure frac is wrapped within [0, 1)

        xir = (xi + 1) % self.Ngrid  # Neighbor cell (wrapped)

        g_main = grad_phi[xi]
        g_next = grad_phi[xir]
        return frac * g_main + (1.0 - frac) * g_next

    def solve_poisson(self, rho):
        """Poisson solver in 1D with periodic BC using FFT."""
        delta_l = np.fft.fft(rho - 1.0)
        k_ell   = 2 * np.pi * np.fft.fftfreq(self.Ngrid)
        phi_l = np.zeros_like(k_ell, dtype=np.complex128)

        # electrostatics => factor C = -1
        C = -1.0
        # avoid divide-by-zero
        phi_l[1:] = (C / (self.Ngrid**2)) * delta_l[1:] / (np.sin(k_ell[1:] / 2)**2)
        phi_l[0]  = 0.0

        phi_x = np.fft.ifft(phi_l).real
        return phi_x

    def central_difference(self, y):
        """Compute derivative via central difference."""
        return 0.5 * (np.roll(y, -1) - np.roll(y, 1))

    def accelerations_1D(x: np.ndarray, m: float, C: float) -> np.ndarray:
        """
        Compute 1D gravitational accelerations for labeled particles,
        ensuring zero net force by subtracting the mass-weighted average acceleration.
        C is the constant in the Poisson equation nabla^2 Phi = C rho
        So for gravity C would be 4 pi G 
        
        *Experimental* not tested yet.
        
        Parameters
        ----------
        x : np.ndarray of shape (N,)
            Positions of particles in label order (not necessarily sorted).
        m : float
            Mass carried by each Lagrangian cell between x[i] and x[i+1].
        C : float
            Gravitational constant factor (e.g., 4*pi*G).
            
        Returns
        -------
        a : np.ndarray of shape (N,)
            The gravitational acceleration at each particle,
            shifted so the total force is zero.
        """
        # 1. Compute differences between consecutive particle positions
        dx = x[1:] - x[:-1]  # shape (N-1,)
        
        # 2. Piecewise densities (could be negative if dx<0),
        #    but for magnitude-based mass we use abs(dx) below
        rho = m / dx  # shape (N-1,)
        
        # 3. Segment mass = rho * |dx|, effectively ±m
        segment_mass = rho * np.abs(dx)
        
        # 4. Cumulative sum to get total mass to the left for each particle
        mass_left = np.cumsum(segment_mass)
        # Prepend zero for the first particle
        mass_left = np.concatenate(([0.0], mass_left), axis=0)  # shape (N,)
        
        # 5. Gravitational acceleration from mass_left
        a = -C * mass_left
        
        # 6. Subtract mean mass-weighted acceleration to ensure net force = 0
        N = x.shape[0]
        total_mass = N * m  # sum of all cell masses if each cell is mass m
        a_mean = np.sum(m * a) / total_mass
        a -= a_mean
        
        return a

    def single_step(self):
        """Advance one time step with a leapfrog ."""
        dt = self.dt

        # 1) Drift half-step
        self.xp += 0.5 * dt * self.vp
        self.xe += 0.5 * dt * self.ve
        self.array_periodic_boundary(self.xp)
        self.array_periodic_boundary(self.xe)

        # 2) Deposit
        rhop = self.CIC_deposit(self.xp, weight=self.chargeIon)
        rhoe = self.CIC_deposit(self.xe, weight=self.chargeE)
        rho  = rhop + rhoe
        rho -= rho.mean()

        # 3) Solve Poisson
        phi = self.solve_poisson(rho)

        # 4) Compute grad
        grad_phi = self.central_difference(phi) * self.Ngrid  # multiply by dx^-1 

        # 5) Gather => acceleration
        ap =  (self.chargeIon / self.massIon) * self.rebin_grad_phi(self.xp, grad_phi)
        ae =  (self.chargeE   / self.massE  ) * self.rebin_grad_phi(self.xe, grad_phi)

        # 6) Kick step
        self.vp += dt * ap
        self.ve += dt * ae

        # 7) Final drift half-step
        self.xp += 0.5 * dt * self.vp
        self.xe += 0.5 * dt * self.ve

        # 8) Update time
        self.step_count += 1
        self.time += dt

    def run(self, Nsteps, store_interval=10,snapshots=[]):
        """
        Run the simulation for Nsteps. Return a list of snapshots,
        each a dict with arrays for xp, vp, xe, ve, plus time and step.
        """

        for step in range(Nsteps):
            self.single_step()

            # If we want to store a snapshot:
            if (step % store_interval) == 0:
                self.array_periodic_boundary(self.xp)
                self.array_periodic_boundary(self.xe)
                snapshots.append({
                    "step": self.step_count,
                    "time": self.time,
                    "xp": self.xp.copy(),
                    "vp": self.vp.copy(),
                    "xe": self.xe.copy(),
                    "ve": self.ve.copy()
                })

        return snapshots

def plot_phase_space(sim, s, vmax=1):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import numpy as np

    # ------------------------------
    # 1. Precompute Histogram Parameters
    # ------------------------------
    
    # Define the number of bins for histograms
    nbins_x = sim.Ngrid  # Number of bins for x-axis histogram
    nbins_y = sim.Ngrid  # Number of bins for y-axis histogram

    # Define the fixed ranges for x and y axes
    x_min, x_max = 0, 1        # Position range
    y_min, y_max = -vmax, vmax # Velocity range

    # Initialize variables to store maximum counts across all frames
    max_count_x_e = 10  # Maximum count for x-axis histogram (Electrons)
    max_count_x_p = 10  # Maximum count for x-axis histogram (Protons)
    max_count_y_e = 10  # Maximum count for y-axis histogram (Electrons)
    max_count_y_p = 10  # Maximum count for y-axis histogram (Protons)

    # Iterate through all frames to determine maximum histogram counts
    for frame in s:
        # Electrons x-data
        x_data_e = frame["xe"]
        counts_x_e, _ = np.histogram(x_data_e, bins=nbins_x, range=(x_min, x_max))
        max_count_x_e = max(max_count_x_e, counts_x_e.max())

        # Protons x-data
        x_data_p = frame["xp"]
        counts_x_p, _ = np.histogram(x_data_p, bins=nbins_x, range=(x_min, x_max))
        max_count_x_p = max(max_count_x_p, counts_x_p.max())

        # Electrons y-data
        y_data_e = frame["ve"]
        counts_y_e, _ = np.histogram(y_data_e, bins=nbins_y, range=(y_min, y_max))
        max_count_y_e = max(max_count_y_e, counts_y_e.max())

        # Protons y-data
        vp_scaled = np.sqrt(sim.massIon / sim.massE) * frame["vp"]
        y_data_p = vp_scaled
        counts_y_p, _ = np.histogram(y_data_p, bins=nbins_y, range=(y_min, y_max))
        max_count_y_p = max(max_count_y_p, counts_y_p.max())

    # ------------------------------
    # 2. Create Subplots with Marginals
    # ------------------------------
    
    # Create a 2x2 subplot grid
    fig = make_subplots(
        rows=2, cols=2,
        shared_xaxes=True,
        shared_yaxes=True,
        vertical_spacing=0.05,
        horizontal_spacing=0.05,
        specs=[
            [None, {"type": "histogram"}],        # Top-Left: Empty, Top-Right: X Histogram
            [{"type": "histogram"}, {"type": "scatter"}]  # Bottom-Left: Y Histogram, Bottom-Right: Scatter Plot
        ],
        column_widths=[0.2, 0.8],  # Allocate 20% width to histograms, 80% to scatter plot
        row_heights=[0.2, 0.8]     # Allocate 20% height to top histograms, 80% to scatter plot
    )

    # ------------------------------
    # 3. Initialize Traces for the First Frame
    # ------------------------------
    
    # Select the initial frame
    i = 0
    frame = s[i]
    xe = frame["xe"]
    ve = frame["ve"]
    xp = frame["xp"]
    vp = frame["vp"]
    vp_scaled = np.sqrt(sim.massIon / sim.massE) * vp

    # Add scatter trace for electrons
    scatter_e = go.Scatter(
        x=xe,
        y=ve,
        mode='markers',
        name=f"Electrons t={frame['time']:.1f} i={frame['step']}",
        marker=dict(color="blue", size=5, opacity=0.6),
        showlegend=True
    )
    fig.add_trace(scatter_e, row=2, col=2)  # Bottom-Right subplot

    # Add scatter trace for protons
    scatter_p = go.Scatter(
        x=xp,
        y=vp_scaled,
        mode='markers',
        name="Protons",
        marker=dict(color="red", size=5, opacity=0.8),
        showlegend=True
    )
    fig.add_trace(scatter_p, row=2, col=2)  # Bottom-Right subplot

    # Add histogram trace for electrons (x-axis) (Top-Right subplot)
    hist_x_e = go.Histogram(
        x=xe,
        nbinsx=nbins_x,
        marker_color='blue',
        opacity=0.5,
        name='Electrons',
        showlegend=True,
        xbins=dict(start=x_min, end=x_max, size=(x_max - x_min) / nbins_x)
    )
    fig.add_trace(hist_x_e, row=1, col=2)  # Top-Right subplot

    # Add histogram trace for protons (x-axis) (Top-Right subplot)
    hist_x_p = go.Histogram(
        x=xp,
        nbinsx=nbins_x,
        marker_color='red',
        opacity=0.5,
        name='Protons',
        showlegend=True,
        xbins=dict(start=x_min, end=x_max, size=(x_max - x_min) / nbins_x)
    )
    fig.add_trace(hist_x_p, row=1, col=2)  # Top-Right subplot

    # Add histogram trace for electrons (y-axis) (Bottom-Left subplot)
    hist_y_e = go.Histogram(
        y=ve,
        nbinsy=nbins_y,
        marker_color='blue',
        opacity=0.5,
        name='Electrons',
        showlegend=False,
        ybins=dict(start=-vmax, end=vmax, size=2 * vmax / nbins_y)
    )
    fig.add_trace(hist_y_e, row=2, col=1)  # Bottom-Left subplot

    # Add histogram trace for protons (y-axis) (Bottom-Left subplot)
    hist_y_p = go.Histogram(
        y=vp_scaled,
        nbinsy=nbins_y,
        marker_color='red',
        opacity=0.5,
        name='Protons',
        showlegend=False,
#        ybins=dict(start=-vmax, end=vmax, size=2 * vmax / nbins_y)
        xbins=dict(start=-vmax, end=vmax, size=2 * vmax / nbins_y)
    )
    fig.add_trace(hist_y_p, row=2, col=1)  # Bottom-Left subplot

    # ------------------------------
    # 4. Configure Slider Steps
    # ------------------------------
    
    steps = []
    for i in range(len(s)):
        frame = s[i]
        xe = frame["xe"]
        ve = frame["ve"]
        xp = frame["xp"]
        vp = frame["vp"]
        vp_scaled = np.sqrt(sim.massIon / sim.massE) * vp

        # Combine data for histograms
        x_data_e = xe
        x_data_p = xp
        y_data_e = ve
        y_data_p = vp_scaled

        step = dict(
            method="update",
            args=[
                {
                    # Update scatter traces
                    "x": [xe, xp, x_data_e, x_data_p, None, None],        # Trace 0: scatter_e x, Trace 1: scatter_p x, Trace 2: hist_x_e x, Trace 3: hist_x_p x, Trace 4: hist_y_e x (unused), Trace 5: hist_y_p x (unused)
                    "y": [ve, vp_scaled, None, None, y_data_e, y_data_p] # Trace 0: scatter_e y, Trace 1: scatter_p y, Trace 2: hist_x_e y (unused), Trace 3: hist_x_p y (unused), Trace 4: hist_y_e y, Trace 5: hist_y_p y
                },
                {
                    "title": f"Phase Space: t={frame['time']:.1f} i={frame['step']}"
                }
            ],
            label=f"t={frame['time']:.1f} i={frame['step']}"
        )
        steps.append(step)

    # Define the slider
    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Time = "},
        pad={"t": 50},
        steps=steps
    )]

    # ------------------------------
    # 5. Finalize Layout
    # ------------------------------
    
    fig.update_layout(
        sliders=sliders,
        title="Phase Space",
        showlegend=True,
        legend=dict(title="Legend", orientation="v", x=1, y=1),
        template="plotly_white",
        height=700,
        width=800,
        barmode='overlay'  # Overlay histograms for better visibility
    )

    # Configure axes for histograms and scatter plot

    # Scatter Plot Axes (Bottom-Right subplot: row=2, col=2)
    fig.update_xaxes(title_text="x", range=[x_min, x_max], row=2, col=2)
    fig.update_yaxes(title_text="v", range=[y_min, y_max], row=2, col=2)

    # X-axis Histogram (Top-Right subplot: row=1, col=2)
    fig.update_xaxes(title_text="x Distribution", range=[x_min, x_max], row=1, col=2)
    fig.update_yaxes(title_text="Counts", range=[0, max(max_count_x_e, max_count_x_p)], row=1, col=2)

    # Y-axis Histogram (Bottom-Left subplot: row=2, col=1)
    fig.update_xaxes(title_text="Counts", range=[0, max(max_count_y_e, max_count_y_p)], row=2, col=1, showticklabels=False)
    fig.update_yaxes(title_text="v Distribution", range=[y_min, y_max], row=2, col=1)

    # Hide unused subplot (Top-Left)
    fig.update_layout(
        annotations=[
            dict(
                text="",
                x=0.15, y=0.85,
                showarrow=False,
                xref="paper",
                yref="paper"
            )
        ]
    )

    # ------------------------------
    # 6. Display the Figure
    # ------------------------------
    
    fig.show()

In [ ]:
# Setup initial data : 2 particles orbiting
Np = 2
xp0 = np.array([0.5])
vp0 = 0
xe0 = np.array([0.6])
ve0 = 0


# Create the simulation
osim = PICSimulation(
    xp=xp0, vp=vp0,
    xe=xe0, ve=ve0,
    Ngrid    = 128,
    dt       = 0.003,
    massIon  = 1.0,
    massE    = 1e-1,
    chargeIon=  1.0,
    chargeE  = -1.0
)

so=0.
so = [] # store snapshots
# Run for 100 steps, storing snapshots every 2 steps
so = osim.run(Nsteps=400, store_interval=4)
#so = osim.run(Nsteps=30_000, store_interval=1_000,snapshots=so)

print(f"Final time:  {osim.time:.2f}, Final step: {osim.step_count}")
print("Number of snapshots:", len(so))

plot_phase_space(osim,so,vmax=2)

In [ ]:
# Setup initial data: random proton and electron positions
Np = 1000
xp0 = np.random.rand(Np)
vp0 = np.random.randn(Np)*0.1
xe0 = np.random.rand(Np)
ve0 = np.random.randn(Np)*0.1

vp0 -= vp0.mean()
ve0 -= ve0.mean()

# Create the simulation
sim = PICSimulation(
    xp=xp0, vp=vp0,
    xe=xe0, ve=ve0,
    Ngrid    = 128,
    dt       = 0.003,
    massIon  = 1.0,
    massE    = 1e-1,
    chargeIon=  1.0,
    chargeE  = -1.0
)

s = [] # store snapshots
# Run for 100 steps, storing snapshots every 2 steps
s = sim.run(Nsteps=100, store_interval=2)
s = sim.run(Nsteps=30_000, store_interval=1_000,snapshots=s.copy())

print(f"Final time:  {sim.time:.2f}, Final step: {sim.step_count}")
print("Number of snapshots:", len(s))

plot_phase_space(sim,s,vmax=0.5)

In [ ]:
# initial conditions: random electron positions
Nh = 800
xp0 = np.random.uniform(size=Nh)  # initial positions  ions
xp0 = np.linspace(1/Nh/2, 1-1/Nh/2, Nh) # uniform lattice of protons
vp0 = np.zeros(shape=Nh)          # initial velocities
xe0 = np.random.uniform(size=Nh)  # same for electrons
ve0 = np.zeros(shape=Nh)



# Create the simulation
sim = PICSimulation(
    xp=xp0, vp=vp0,
    xe=xe0, ve=ve0,
    Ngrid    = 128,
    dt       = 0.003,
    massIon  = 1.0,
    massE    = 1e-1,
    chargeIon=  1.0,
    chargeE  = -1.0
)

s = [] # store snapshots
# Run for 100 steps, storing snapshots every 2 steps
s = sim.run(Nsteps=100, store_interval=4)
s = sim.run(Nsteps=30_000, store_interval=1_000,snapshots=s)

print(f"Final time:  {sim.time:.2f}, Final step: {sim.step_count}")
print("Number of snapshots:", len(s))

plot_phase_space(sim,s,vmax=0.5)


In [ ]:
import matplotlib.pyplot as plt
Nout = len(s)
relE = np.array([[s[i]["time"], s[i]["vp"].var()/s[i]["ve"].var() * sim.massIon/sim.massE] for i in range(1,Nout)])
plt.plot(relE[:,0], relE[:,1],'.');

In [ ]:
# initial conditions: Large velocity perturbation in the electrons
Nh = 800
xp0 = np.random.uniform(size=Nh)  # initial positions  ions
xp0 = np.linspace(1/Nh/2, 1-1/Nh/2, Nh) # uniform lattice of protons
vp0 = np.zeros(shape=Nh)          # initial velocities
xe0 = np.linspace(1/Nh/2, 1-1/Nh/2, Nh) # same for electrons
ve0 = 0.1 * np.sin(8 * 2*np.pi*xe0)



# Create the simulation
sim = PICSimulation(
    xp=xp0, vp=vp0,
    xe=xe0, ve=ve0,
    Ngrid    = 128,
    dt       = 0.003,
    massIon  = 1.0,
    massE    = 1e-1,
    chargeIon=  1.0,
    chargeE  = -1.0
)

s = [] # store snapshots
# Run for 100 steps, storing snapshots every 2 steps
s = sim.run(Nsteps=400, store_interval=4)
s = sim.run(Nsteps=30_000, store_interval=1_000,snapshots=s)

print(f"Final time:  {sim.time:.2f}, Final step: {sim.step_count}")
print("Number of snapshots:", len(s))

plot_phase_space(sim,s,vmax=0.5)


In [ ]:
# initial conditions: Two stream instability
Nh = 2048
xp0 = np.random.uniform(size=Nh)  # initial positions  ions
xp0 = np.linspace(1/Nh/2, 1-1/Nh/2, Nh) # uniform lattice of protons
vs = 0.02 
vp0 = vs * np.ones(shape=Nh)          # initial velocities
xe0 = np.linspace(1/Nh/2, 1-1/Nh/2, Nh) # same for electrons
mi = 1 ; me = 1
ve0 =  0.01 * np.sin(8 * 2*np.pi*xe0) - vs * mi/me 

# Create the simulation
sim = PICSimulation(
    xp=xp0, vp=vp0,
    xe=xe0, ve=ve0,
    Ngrid    = 256,
    dt       = 0.003,
    massIon  = mi,
    massE    = me,
    chargeIon=  1.0,
    chargeE  = -1.0
)

s = [] # store snapshots
# Run for 100 steps, storing snapshots every 2 steps
s = sim.run(Nsteps=16_400, store_interval=24)
#s = sim.run(Nsteps=30_000, store_interval=1_000,snapshots=s)

print(f"Final time:  {sim.time:.2f}, Final step: {sim.step_count}")
print("Number of snapshots:", len(s))

plot_phase_space(sim,s,vmax=2.5 * mi/me *vs)


Final time:  49.20, Final step: 16400
Number of snapshots: 684


In [ ]:
# initial conditions: uniform electron positions
Nh = 10
xp0 = np.random.uniform(size=Nh)  # initial positions  ions
xp0 = np.linspace(1/Nh/2, 1-1/Nh/2, Nh) # uniform lattice of protons
vp0 = np.zeros(shape=Nh)          # initial velocities
xe0 = np.linspace(0, 1-1/Nh, Nh)  # same for electronsa offset half a lattice spacing
ve0 = np.zeros(shape=Nh)


# Create the simulation
sim = PICSimulation(
    xp=xp0, vp=vp0,
    xe=xe0, ve=ve0,
    Ngrid    = 128,
    dt       = 0.03,
    massIon  = 1.0,
    massE    = 1e-1,
    chargeIon=  1.0,
    chargeE  = -1.0
)

s = [] # store snapshots
# Run for 100 steps, storing snapshots every 2 steps
s = sim.run(Nsteps=100, store_interval=1)
#s = sim.run(Nsteps=30_000, store_interval=1_000,snapshots=s)

print(f"Final time:  {sim.time:.2f}, Final step: {sim.step_count}")
print("Number of snapshots:", len(s))

plot_phase_space(sim,s,vmax=0.5)


In [12]:
# initial conditions: uniform electron positions
Nh = 5
xp0 = np.random.uniform(size=Nh)  # initial positions  ions
xp0 = np.linspace(1/Nh/2, 1-1/Nh/2, Nh) # uniform lattice of protons
vp0 = np.zeros(shape=Nh)          # initial velocities
xe0 = np.linspace(0, 1-1/Nh, Nh)  # same for electronsa offset half a lattice spacing
ve0 = np.zeros(shape=Nh)

# Create the simulation
sim = PICSimulation(
    xp=xp0, vp=vp0,
    xe=xe0, ve=ve0,
    Ngrid    = 10,
    dt       = 0.03,
    massIon  = 1.0,
    massE    = 1e-1,
    chargeIon=  1.0,
    chargeE  = -1.0
)

In [ ]:
r = sim.CIC_deposit(xe0)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r,'x-')

## Beware this code may need more testing.
